In [8]:
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim.downloader as api
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.neighbors import KNeighborsClassifier
from google.colab import files
import io
import matplotlib.pyplot as plt
from gensim.models import FastText
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:

punct =['.',',',';',':','!','\'', '?', '"', '(', ')', '[', ']', '<', '>', '\\', '/']
english_stop_words = stopwords.words('english')
lemmatizer = nltk.stem.WordNetLemmatizer()

from google.colab import drive
drive.mount('/content/drive')

words = ['hood', 'java', 'mole', 'pitcher', 'pound', 'seal', 'spring', 'square', 'trunk', 'yard']

def get_list_tokens(string):
	sentence_split = nltk.tokenize.sent_tokenize(string)
	list_tokens = []
	for sentence in sentence_split:
		list_tokens_sentence = nltk.tokenize.word_tokenize(sentence)
		for token in list_tokens_sentence:
			list_tokens.append(lemmatizer.lemmatize(token).lower())

	return list_tokens

def remove_stop_words(list_tokens):
	clean_list_tokens = []
	for token in list_tokens:
		if token not in english_stop_words:
			clean_list_tokens.append(token)

	return clean_list_tokens

def remove_punct(list_tokens):
	no_punct = []
	for i in list_tokens:
		i = re.sub(r'[^\w\s]','',i)
		if i:
			no_punct.append(i)
	# no_punct = [i for i in list_tokens if i not in punct]
	return no_punct

def convertTow2v(list_tokens, selected_features, w2v_vector):
		w2v = np.zeros((len(list_tokens), len(selected_features), 300))
		set_s_f = set(selected_features)
		for i in range(len(list_tokens)):
			# in case some training data tokens are not in features
			inters = set(list_tokens[i]).intersection(set_s_f)
			for word in inters:
				j = selected_features.index(word)
				w2v[i][j] = w2v_vector[j]

		w2v = np.sum(w2v, axis = 1)

		return w2v




# df = pd.DataFrame(stats_all, columns=['accuracy', 'precision', 'recall', 'fscore'], index=words)
# print(df)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
model2 = api.load('word2vec-google-news-300')

[=================================================-] 99.8% 1659.4/1662.8MB downloaded


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [12]:
stats_all = []
for word in words:
    
    # train_text = files.upload()
    # train_label = files.upload()
    # test_text = files.upload()
    # test_label = files.upload()

    train_text = pd.read_csv('/content/drive/My Drive/CoarseWSD_P2/{}/train.data.txt'.format(word),
                             sep='\t',
                             names=['index', 'sentence'])
    train_label = pd.read_csv('/content/drive/My Drive/CoarseWSD_P2/{}/train.gold.txt'.format(word),
                              sep='\t',
                              names=['label'])

    test_text = pd.read_csv('/content/drive/My Drive/CoarseWSD_P2/{}/test.data.txt'.format(word),
                            sep='\t',
                            names=['index', 'sentence'])
    test_label = pd.read_csv('/content/drive/My Drive/CoarseWSD_P2/{}/test.gold.txt'.format(word),
                             sep='\t',
                             names=['label'])


    # merge train date with labels data in one table
    train = pd.merge(train_text, train_label, left_index=True, right_index=True)
    test = pd.merge(test_text, test_label, left_index=True, right_index=True)


    # preprocess the sentences
    list_tokens = []
    for i in range(0, len(train.sentence)):
        sentence_tokens = get_list_tokens(train.sentence[i])
        rm_st = remove_stop_words(sentence_tokens)
        rm_punct = remove_punct(rm_st)
        list_tokens.append(rm_punct)
        # update cell
        train.at[i,'sentence']=' '.join(rm_punct)

    test_tokens = []
    for i in range(0, len(test.sentence)):
        sentence_tokens = get_list_tokens(test.sentence[i])
        rm_st = remove_stop_words(sentence_tokens)
        rm_punct = remove_punct(rm_st)
        test_tokens.append(rm_punct)
        # update cell
        test.at[i,'sentence']=' '.join(rm_punct)


    list_rows = train['sentence']
    vectorizer = TfidfVectorizer(max_features=1000)
    X = vectorizer.fit(list_rows)

    selected_features = vectorizer.get_feature_names()


    # Size is the number of dimensions of the embeddings we are going to learn
    # Window is the size considered for context of a target word
    # Min count is the minimum number of times that a word need to occur to be learnt
    model = Word2Vec(list_tokens, size=300, window=5, min_count=1)
    # model = api.load('word2vec-google-news-300')
    # model = FastText(size=100, window=3, min_count=1)
    # model.build_vocab(sentences=list_tokens)
    model.train(list_tokens, total_examples=len(list_tokens), epochs=100)
    # model.train(list_tokens)
    # model = Word2Vec.load('word2vec-google-news-300')
    not_in_model = model2.doesnt_match(selected_features)
    # print(len(not_in_model))

    # selected_features = selected_features - not_in_model
    w2v_vector = [model2[feature] if feature in not_in_model else model[feature] for feature in selected_features ]
    # w2v_vector = model[selected_features]


    w2v = convertTow2v(list_tokens, selected_features, w2v_vector)
    w2v_test = convertTow2v(test_tokens, selected_features, w2v_vector)

    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(w2v, train.label)

    # for sentence in w2v_test:
    prediction = knn.predict(w2v_test)

    accuracy = accuracy_score(test.label, prediction)
    precision, recall, fscore, support = precision_recall_fscore_support(test.label, prediction, average='macro', zero_division=0)
    stats = [accuracy, precision, recall, fscore]
    stats_all.append(stats)


    print(word)
    print(stats)

df = pd.DataFrame(stats_all, columns=['accuracy', 'precision', 'recall', 'fscore'], index=words)
print(df)


/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:70: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


hood
[0.6219512195121951, 0.4839406207827261, 0.4818231413976095, 0.48191846708316416]


/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:70: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


java
[0.9823742871954381, 0.9810062471203621, 0.9819358014075263, 0.9814661379620608]


/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:70: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


...

square
[0.8792270531400966, 0.9027573529411765, 0.7255825699619776, 0.7768634395503282]


/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:70: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


trunk
[0.6753246753246753, 0.5401382243487507, 0.4846757852077001, 0.49538461538461537]


/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:70: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


yard
[0.8055555555555556, 0.42028985507246375, 0.47540983606557374, 0.4461538461538461]
         accuracy  precision    recall    fscore
hood     0.621951   0.483941  0.481823  0.481918
java     0.982374   0.981006  0.981936  0.981466
mole     0.762136   0.760831  0.702242  0.725466
pitcher  0.993260   0.623219  0.613781  0.618308
pound    0.793814   0.523236  0.531034  0.524510
seal     0.840220   0.748007  0.708796  0.722212
spring   0.864333   0.867710  0.829832  0.845906
square   0.879227   0.902757  0.725583  0.776863
trunk    0.675325   0.540138  0.484676  0.495385
yard     0.805556   0.420290  0.475410  0.446154


<bound method Figure.show of <Figure size 2160x2160 with 10 Axes>>
None
